In [21]:
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader


# 加载模型
model = SentenceTransformer('nghuyong/ernie-3.0-nano-zh')

No sentence-transformers model found with name C:\Users\Administrator/.cache\torch\sentence_transformers\nghuyong_ernie-3.0-nano-zh. Creating a new one with MEAN pooling.
Some weights of ErnieModel were not initialized from the model checkpoint at C:\Users\Administrator/.cache\torch\sentence_transformers\nghuyong_ernie-3.0-nano-zh and are newly initialized: ['ernie.pooler.dense.weight', 'ernie.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
trash = {
    '教室内': {
        '可回收垃圾': ['纸张', '粉笔盒', '笔壳', '书', '作业本'],
        '其他垃圾': ['粉笔灰', '粉笔头', '铅笔屑', '小铅笔', '用过的笔芯', '橡皮屑', '灰尘', '用过的纸巾', '修正带'],
        '有害垃圾': ['涂改液', '胶水', '胶棒', '胶带', '打印机墨盒', '打印机色带', '过期药品', '废电池', '废油漆', '废荧光灯管', '废矿物油', '废农药', '废杀虫剂', '废水银温度计', '废水银血压计']
    },
    '办公室内': {
        '可回收垃圾': ['纸张', '矿泉水瓶', '报纸', '杂志', '快递盒', '牛奶盒', '包装盒', '纸盒', '易拉罐', '塑料瓶', '塑料袋', '塑料碗', '塑料盒', '塑料玩具', '塑料衣架', '塑料杯', '塑料餐具', '塑料文件袋', '塑料文件夹'],
        '其他垃圾': ['用完的胶水盒', '润喉糖包装纸', '用过的笔芯', '用过的纸巾', '修正带', '粉笔灰', '粉笔头', '铅笔屑', '铅笔', '橡皮屑', '灰尘', '废弃的打火机', '废弃的香水瓶', '废弃的口红', '废弃的指甲油', '废弃的眼影', '废弃的粉饼', '废弃的粉底液', '废弃的睫毛膏', '废弃的眉笔'],
        '易腐垃圾': ['茶叶渣', '水果核', '水果皮', '植物叶', '植物枝', '植物根', '剩菜剩饭', '骨头', '蛋壳', '花卉', '绿植'],
    },
    '医务室内': {
        '可回收垃圾': ['药物盒'],
        '其他垃圾': ['消毒棉球', '退热贴', '棉棒', '废弃创可贴', '胶带', '用过的纸巾'],
        '有害垃圾': ['废弃水银温度计', '废弃水银血压计', '废弃药品', '废弃药片', '废弃药瓶', '废弃药膏']
    },
    '打印室内': {
        '可回收垃圾': ['废弃纸张', '纸盒'],
        '有害垃圾': ['打印机墨盒', '硒鼓', '废电池']
    },
    '医务室内': {
        '可回收垃圾': ['药物盒'],
        '其他垃圾': ['消毒棉球', '退热贴', '棉棒', '废弃创可贴', '胶带', '用过的纸巾'],
        '有害垃圾': ['废弃水银温度计']
    },
    '卫生间内': {
        '其他垃圾': ['用过的纸巾', '用过的卫生用品']
    },
    '校园内': {
        '可回收垃圾': ['废弃球类', '羽毛球拍', '金属', '废金属', '废旧金属', '铁', '铜', '铝', '锌', '锡', '镍', '铅', '钢', '合金'],
        '其他垃圾': ['灰尘'],
        '易腐垃圾': ['落叶', '树枝'],
        '有害垃圾': ['废弃电池', '污水']
    },
    '食堂内': {
        '可回收垃圾': ['牛奶盒', '纸盒', '易拉罐'],
        '其他垃圾': ['用过的纸巾', '塑料袋'],
        '易腐垃圾': ['剩菜剩饭', '骨头', '果皮', '蛋壳']
    },
    '宿舍内': {
        '可回收垃圾': ['用过的牙膏管', '纸盒', '纸张', '旧衣物', '塑料瓶'],
        '其他垃圾': ['灰尘', '废弃牙刷'],
    }
}

In [23]:
trash_type = {}
for k1, v1 in trash.items():
    for k2, v2 in v1.items():
        if k2 not in trash_type:
            trash_type[k2] = []
        for v3 in v2:
            trash_type[k2].append(v3.strip())
trash_type

{'可回收垃圾': ['纸张',
  '粉笔盒',
  '笔壳',
  '书',
  '作业本',
  '纸张',
  '矿泉水瓶',
  '报纸',
  '杂志',
  '快递盒',
  '牛奶盒',
  '包装盒',
  '纸盒',
  '易拉罐',
  '塑料瓶',
  '塑料袋',
  '塑料碗',
  '塑料盒',
  '塑料玩具',
  '塑料衣架',
  '塑料杯',
  '塑料餐具',
  '塑料文件袋',
  '塑料文件夹',
  '药物盒',
  '废弃纸张',
  '纸盒',
  '废弃球类',
  '羽毛球拍',
  '金属',
  '废金属',
  '废旧金属',
  '铁',
  '铜',
  '铝',
  '锌',
  '锡',
  '镍',
  '铅',
  '钢',
  '合金',
  '牛奶盒',
  '纸盒',
  '易拉罐',
  '用过的牙膏管',
  '纸盒',
  '纸张',
  '旧衣物',
  '塑料瓶'],
 '其他垃圾': ['粉笔灰',
  '粉笔头',
  '铅笔屑',
  '小铅笔',
  '用过的笔芯',
  '橡皮屑',
  '灰尘',
  '用过的纸巾',
  '修正带',
  '用完的胶水盒',
  '润喉糖包装纸',
  '用过的笔芯',
  '用过的纸巾',
  '修正带',
  '粉笔灰',
  '粉笔头',
  '铅笔屑',
  '铅笔',
  '橡皮屑',
  '灰尘',
  '废弃的打火机',
  '废弃的香水瓶',
  '废弃的口红',
  '废弃的指甲油',
  '废弃的眼影',
  '废弃的粉饼',
  '废弃的粉底液',
  '废弃的睫毛膏',
  '废弃的眉笔',
  '消毒棉球',
  '退热贴',
  '棉棒',
  '废弃创可贴',
  '胶带',
  '用过的纸巾',
  '用过的纸巾',
  '用过的卫生用品',
  '灰尘',
  '用过的纸巾',
  '塑料袋',
  '灰尘',
  '废弃牙刷'],
 '有害垃圾': ['涂改液',
  '胶水',
  '胶棒',
  '胶带',
  '打印机墨盒',
  '打印机色带',
  '过期药品',
  '废电池',
  '废油漆',
  '废荧光灯管',
  '废矿物油',
  '废农药',
  '废杀虫剂',
  

In [24]:
# 加载数据，构建训练集

# # 读取文件
# f = open('./cls.txt', 'r', encoding='utf-8').readlines()
# f = [i.strip() for i in f]

# # 将数据转换为字典
# data = {}
# for i in f:
#     t, e = i.split('：')
#     e = e.split('', '')
#     data[t] = e
#     print(len(e))

# 构建训练集：将每个类别与其中的元素相似度定义为1，与其他类别的元素相似度定义为0
train_data = []
for k1, v1 in trash_type.items():
    for v in v1:
        train_data.append(InputExample(texts=[k1, v], label=float(1)))
    for k2, v2 in trash_type.items():
        if k1 != k2:
            for v in v2:
                train_data.append(InputExample(texts=[k1, v], label=float(0)))
print(len(train_data))
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)

516


In [25]:
from sentence_transformers import losses

train_loss = losses.ContrastiveLoss(model)  # 定义损失函数，与查询时使用的方法一致
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=50)  # 训练模型
model.save('./model/')

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

Iteration:   0%|          | 0/33 [00:00<?, ?it/s]

In [26]:
from sentence_transformers import util
import torch


corpus = ['其他垃圾', '可回收垃圾', '易腐垃圾', '有害垃圾']
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['电池', '外卖盒', '骨头', '化妆品', '纸箱']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))
        break





Query: 电池

Top 5 most similar sentences in corpus:
有害垃圾 (Score: 0.7101)




Query: 外卖盒

Top 5 most similar sentences in corpus:
可回收垃圾 (Score: 0.9514)




Query: 骨头

Top 5 most similar sentences in corpus:
易腐垃圾 (Score: 0.7340)




Query: 化妆品

Top 5 most similar sentences in corpus:
可回收垃圾 (Score: 0.7763)




Query: 纸箱

Top 5 most similar sentences in corpus:
可回收垃圾 (Score: 0.9138)


In [31]:
import numpy as np
def softmax(x):
    x = [i.cpu().numpy() for i in x]
    return np.exp(x) / np.sum(np.exp(x), axis=0)
def cls(query):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=1)  # topk结果，如果垃圾分类多的话也可以返回多个结果推荐
    result_id, result_score = [], []
    for score, idx in zip(top_results[0], top_results[1]):
        result_id.append(idx)
        result_score.append(score)
    result_score = softmax(result_score)
    result_label = [corpus[i] for i in result_id]
    return result_label[0]
    result = dict(zip(result_label, result_score))
    return result

In [32]:
cls('电池')

'有害垃圾'

In [33]:
c, n = 0, 0
for k, v in trash_type.items():
    for i in v:
        result = cls(i)
        if k == result:
            c += 1
        n += 1
print(c / n)

0.9457364341085271
